In [2]:
import os

# Set dataset path
data_dir = "/kaggle/input/plantdisease/PlantVillage"

# Optional: check if the path exists
print(os.path.exists(data_dir))  # Should print True


True


In [3]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),#tensor is like a multi-dimensional array (matrix) that PyTorch can do math on.
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [5]:
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [8]:
import torch.nn as nn
import torch.nn.functional as F

# Define a simple CNN
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        # 1st convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling
        # 2nd convolutional layer
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # Fully connected layers
        self.fc1 = nn.Linear(32 * 56 * 56, 128)  # 224->112 after pool->56
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):#self allows you to access all the layers and attributes
        x = self.pool(F.relu(self.conv1(x)))  # Conv1 -> ReLU -> Pool
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 -> ReLU -> Pool
        x = x.view(-1, 32 * 56 * 56)          # Flatten
        x = F.relu(self.fc1(x))               # FC1 -> ReLU
        x = self.fc2(x)                       # FC2 -> output
        return x


In [9]:
num_classes = len(dataset.classes)  # Number of disease categories
model = SimpleCNN(num_classes)
print(model)


SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=100352, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=15, bias=True)
)


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

import torch.optim as optim

criterion = nn.CrossEntropyLoss()       # Loss function for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizer


In [ ]:
num_epochs = 5  # You can increase later if needed

for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()  # Set model to training mode

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()           # Reset gradients
        outputs = model(images)         # Forward pass
        loss = criterion(outputs, labels)  # Calculate loss
        loss.backward()                 # Backpropagation
        optimizer.step()                # Update weights

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")



Epoch [1/5], Loss: 1.0409
Epoch [2/5], Loss: 0.3562


In [ ]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No gradient needed for evaluation
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
